In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
def loadCombinedArray(cases,field):
    data = np.concatenate([np.load('/home/haoyan/Desktop/ML-CFD/dataset/archive/'+dataset+'/'+dataset+'_'+case+'_'+field + '.npy') for case in cases])
    return data
def loadLabels(cases,field):
    data = np.concatenate([np.load('/home/haoyan/Desktop/ML-CFD/dataset/archive/'+'labels/'+case+'_'+field + '.npy') for case in cases])
    return data
def Stra_Rota(tensor,k,epsilon):#Forming non-dimensional strain tensor and rotation tensor
    number = len(tensor)
    s = np.zeros((number,3,3))
    w = np.zeros((number,3,3))
    for i in range(number):
        coef = k[i] / epsilon[i]
        s[i] = coef * (tensor[i] + tensor[i].T)/2
        w[i] = coef * (tensor[i] - tensor[i].T)/2
    return s,w
def Invariants1(s,w):#Forming 5 Invariants according to Pope(1975)
    number = len(s)
    l_1 = np.zeros(number)
    l_2 = np.zeros(number)
    l_3 = np.zeros(number)
    l_4 = np.zeros(number)
    l_5 = np.zeros(number)
    for i in range(number):
        tmp_1 = s[i]@s[i]
        tmp_2 = w[i]@w[i]
        tmp_3 = s[i]@s[i]@s[i]
        tmp_4 = w[i]@w[i]@s[i]
        tmp_5 = w[i]@w[i]@s[i]@s[i]
        l_1[i] = tmp_1.trace()
        l_2[i] = tmp_2.trace()
        l_3[i] = tmp_3.trace()
        l_4[i] = tmp_4.trace()
        l_5[i] = tmp_5.trace()
    return l_1,l_2,l_3,l_4,l_5
def General_Tensors(s,w):#Forming 10 general tensors according to Pope(1975)
    number = len(s)
    T_1 = np.zeros((number,3,3))
    T_2 = np.zeros((number,3,3))
    T_3 = np.zeros((number,3,3))
    T_4 = np.zeros((number,3,3))
    T_5 = np.zeros((number,3,3))
    T_6 = np.zeros((number,3,3))
    T_7 = np.zeros((number,3,3))
    T_8 = np.zeros((number,3,3))
    T_9 = np.zeros((number,3,3))
    T_10 = np.zeros((number,3,3))
    for i in range(number):
        tmp_1 = s[i]@s[i]
        tmp_2 = w[i]@w[i]
        tmp_3 = s[i]@w[i]@w[i]
        tmp_4 = s[i]@s[i]@w[i]@w[i]
        T_1[i] = s[i]
        T_2[i] = s[i]@w[i] - w[i]@s[i]
        T_3[i] = s[i]@s[i] - (1/3) * np.eye(3) * tmp_1.trace() 
        T_4[i] = w[i]@w[i] - (1/3) * np.eye(3) * tmp_2.trace()
        T_5[i] = w[i]@s[i]@s[i] - s[i]@s[i]@w[i]
        T_6[i] = w[i]@w[i]@s[i] + s[i]@w[i]@w[i] - (2/3) * np.eye(3) * tmp_3.trace()
        T_7[i] = w[i]@s[i]@w[i]@w[i] - w[i]@w[i]@s[i]@w[i]
        T_8[i] = s[i]@w[i]@s[i]@s[i] - s[i]@s[i]@w[i]@s[i]
        T_9[i] = w[i]@w[i]@s[i]@s[i] + s[i]@s[i]@w[i]@w[i] - 2/3 * np.eye(3)*tmp_4.trace()
        T_10[i] = w[i]@s[i]@s[i]@w[i]@w[i] - w[i]@w[i]@s[i]@s[i]@w[i]
    return T_1,T_2,T_3,T_4,T_5,T_6,T_7,T_8,T_9,T_10
dataset = 'komega'
cases = ['PHLL_case_0p5','PHLL_case_0p8','PHLL_case_1p0','PHLL_case_1p2','PHLL_case_1p5']

In [30]:
gradU =  loadCombinedArray(case,'gradU')
k = loadCombinedArray(case,'k')
epsilon = loadCombinedArray(case,'epsilon')
s,w = Stra_Rota(gradU,k,epsilon)
I_1,I_2,I_3,I_4,I_5 = Invariants1(s,w) #Invariants//verified
T_1,T_2,T_3,T_4,T_5,T_6,T_7,T_8,T_9,T_10 = General_Tensors(s,w)
I1 = np.column_stack((I_1,I_2,I_3,I_4,I_5))
Tensors = np.zeros((len(k),10,3,3))
for i in range(len(k)):
    Tensors[i][0] = T_1[i]
    Tensors[i][1] = T_2[i]
    Tensors[i][2] = T_3[i]
    Tensors[i][3] = T_4[i]
    Tensors[i][4] = T_5[i]
    Tensors[i][5] = T_6[i]
    Tensors[i][6] = T_7[i]
    Tensors[i][7] = T_8[i]
    Tensors[i][8] = T_9[i]
    Tensors[i][9] = T_10[i]
Labels = loadLabels(cases,'b')
Labels = np.delete(Labels.reshape((len(Labels),9)),[3,6,7],axis=1)
print('Shape of DNS/LES labels array: '+str(Labels.shape))

Shape of DNS/LES labels array: (73754, 6)


In [31]:
indices = np.arange(I1.shape[0])
input_shape = I1.shape[1]

x_train, x_val, y_train, y_val, ind_train, ind_val = train_test_split(I1, Labels, indices, test_size=0.2, random_state=10, shuffle=True)

basis_train = Tensors[ind_train]
basis_val = Tensors[ind_val]

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

print(' ')
print('Training features:')
print(x_train.shape)
print('Training tensor basis:')
print(basis_train.shape)
print('Training labels:')
print(y_train.shape)
print(' ')
print('Validation features:')
print(x_val.shape)
print('Validation tensor basis:')
print(basis_val.shape)
print('Validation labels:')
print(y_val.shape)
print(' ')

 
Training features:
(59003, 5)
Training tensor basis:
(59003, 10, 3, 3)
Training labels:
(59003, 6)
 
Validation features:
(14751, 5)
Validation tensor basis:
(14751, 10, 3, 3)
Validation labels:
(14751, 6)
 


In [32]:
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()

#The model has two inputs, a set of input features with a learned mapping, and the tensor basis layer
input_layer = keras.layers.Input(shape=(input_shape),name ='input_layer')
input_tensor_basis = keras.layers.Input(shape=(10,3,3),name='Tensor_input_layer')

#Hidden layer definition
hidden1 = keras.layers.Dense(30,name='Hidden1', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(input_layer)
hidden2 = keras.layers.Dense(30,name='Hidden2', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden1)
hidden3 = keras.layers.Dense(30,name='Hidden3', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden2)
hidden4 = keras.layers.Dense(30,name='Hidden4', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden3)
hidden5 = keras.layers.Dense(30,name='Hidden5', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden4)
hidden6 = keras.layers.Dense(30,name='Hidden6', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden5)
hidden7 = keras.layers.Dense(30,name='Hidden7', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden6)
hidden8 = keras.layers.Dense(30,name='Hidden8', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden7)


#The layer of gn, which are coefficients for each of the ten Tn
gn = keras.layers.Dense(10,name='gn', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-4), activation = "selu")(hidden8)

#Multiply the gn by Tn, with the output being the anisotropy tensor
shaped = keras.layers.Reshape((10,1,1),name='Shape_for_dot_product')(gn)
merge = keras.layers.Dot(axes=1, name='Dot_product')([shaped,input_tensor_basis])

#Reshape the output anisotropy tensor, and trim out duplicate values (it is a symmetric matrix). The end result is a 6 component vector.
shaped_output = keras.layers.Reshape((9,1),name='Shaped_output')(merge)
trimmed_output1 = keras.layers.Lambda(lambda x : x[:,0])(shaped_output)
trimmed_output2 = keras.layers.Lambda(lambda x : x[:,1])(shaped_output)
trimmed_output3 = keras.layers.Lambda(lambda x : x[:,2])(shaped_output)
trimmed_output4 = keras.layers.Lambda(lambda x : x[:,4])(shaped_output)
trimmed_output5 = keras.layers.Lambda(lambda x : x[:,5])(shaped_output)
trimmed_output6 = keras.layers.Lambda(lambda x : x[:,8])(shaped_output)
merged_output = tf.keras.layers.Concatenate()([trimmed_output1,trimmed_output2,trimmed_output3,trimmed_output4,trimmed_output5,trimmed_output6])


model=keras.Model(inputs=[input_layer, input_tensor_basis], outputs=[merged_output])


optimizer = tf.keras.optimizers.Nadam(learning_rate = 2E-5, clipnorm=1000)
model.compile(optimizer,loss='mse',metrics=['mae', 'mse'])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 5)]          0           []                               
                                                                                                  
 Hidden1 (Dense)                (None, 30)           180         ['input_layer[0][0]']            
                                                                                                  
 Hidden2 (Dense)                (None, 30)           930         ['Hidden1[0][0]']                
                                                                                                  
 Hidden3 (Dense)                (None, 30)           930         ['Hidden2[0][0]']                
                                                                                              

2022-06-23 20:51:34.075479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-23 20:51:34.075505: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-23 20:51:34.075523: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cute): /proc/driver/nvidia/version does not exist
2022-06-23 20:51:34.076297: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


                                                                                                  
 Tensor_input_layer (InputLayer  [(None, 10, 3, 3)]  0           []                               
 )                                                                                                
                                                                                                  
 Dot_product (Dot)              (None, 1, 1, 3, 3)   0           ['Shape_for_dot_product[0][0]',  
                                                                  'Tensor_input_layer[0][0]']     
                                                                                                  
 Shaped_output (Reshape)        (None, 9, 1)         0           ['Dot_product[0][0]']            
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['Shaped_output[0][0]']          
          

In [33]:
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.2,patience=10,verbose=0,mode="auto",min_delta=0.0001,cooldown=0,min_lr=0)
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0,patience=40,verbose=0,mode="auto",baseline=None,restore_best_weights=True)
history = model.fit([x_train,basis_train],y_train,batch_size=1000,epochs=1000,validation_data = ([x_val,basis_val],y_val),verbose=1)

Epoch 1/1000
60/60 [==============================] - 3s 13ms/step - loss: 11.2567 - mae: 1.2803 - mse: 11.0217 - val_loss: 8.5244 - val_mae: 1.1627 - val_mse: 8.2894
Epoch 2/1000
60/60 [==============================] - 0s 5ms/step - loss: 6.5608 - mae: 1.0058 - mse: 6.3259 - val_loss: 5.2828 - val_mae: 0.9140 - val_mse: 5.0479
Epoch 3/1000
60/60 [==============================] - 0s 5ms/step - loss: 4.1713 - mae: 0.7926 - mse: 3.9365 - val_loss: 3.4628 - val_mae: 0.7255 - val_mse: 3.2280
Epoch 4/1000
60/60 [==============================] - 0s 6ms/step - loss: 2.7936 - mae: 0.6335 - mse: 2.5588 - val_loss: 2.3527 - val_mae: 0.5821 - val_mse: 2.1180
Epoch 5/1000
60/60 [==============================] - 0s 5ms/step - loss: 1.9289 - mae: 0.5112 - mse: 1.6943 - val_loss: 1.6491 - val_mae: 0.4709 - val_mse: 1.4145
Epoch 6/1000
60/60 [==============================] - 0s 5ms/step - loss: 1.3708 - mae: 0.4149 - mse: 1.1362 - val_loss: 1.1899 - val_mae: 0.3845 - val_mse: 0.9553
Epoch 7/1000


Epoch 51/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2507 - mae: 0.0723 - mse: 0.0174 - val_loss: 0.2507 - val_mae: 0.0722 - val_mse: 0.0174
Epoch 52/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2501 - mae: 0.0710 - mse: 0.0169 - val_loss: 0.2501 - val_mae: 0.0709 - val_mse: 0.0169
Epoch 53/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2496 - mae: 0.0700 - mse: 0.0165 - val_loss: 0.2496 - val_mae: 0.0699 - val_mse: 0.0164
Epoch 54/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2491 - mae: 0.0691 - mse: 0.0160 - val_loss: 0.2491 - val_mae: 0.0691 - val_mse: 0.0160
Epoch 55/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2487 - mae: 0.0682 - mse: 0.0156 - val_loss: 0.2486 - val_mae: 0.0679 - val_mse: 0.0156
Epoch 56/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2483 - mae: 0.0674 - mse: 0.0152 - val_loss: 0.2482 - val_mae: 0.0671 - val_mse: 0.0152
Epoch 57/1

Epoch 101/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2365 - mae: 0.0506 - mse: 0.0083 - val_loss: 0.2362 - val_mae: 0.0500 - val_mse: 0.0081
Epoch 102/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2362 - mae: 0.0504 - mse: 0.0082 - val_loss: 0.2359 - val_mae: 0.0499 - val_mse: 0.0080
Epoch 103/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2360 - mae: 0.0502 - mse: 0.0082 - val_loss: 0.2358 - val_mae: 0.0502 - val_mse: 0.0080
Epoch 104/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2357 - mae: 0.0501 - mse: 0.0081 - val_loss: 0.2355 - val_mae: 0.0496 - val_mse: 0.0079
Epoch 105/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2355 - mae: 0.0500 - mse: 0.0080 - val_loss: 0.2352 - val_mae: 0.0494 - val_mse: 0.0079
Epoch 106/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2352 - mae: 0.0498 - mse: 0.0080 - val_loss: 0.2350 - val_mae: 0.0493 - val_mse: 0.0079
Epoc

60/60 [==============================] - 0s 6ms/step - loss: 0.2185 - mae: 0.0440 - mse: 0.0062 - val_loss: 0.2181 - val_mae: 0.0435 - val_mse: 0.0060
Epoch 151/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2180 - mae: 0.0439 - mse: 0.0061 - val_loss: 0.2176 - val_mae: 0.0434 - val_mse: 0.0060
Epoch 152/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2175 - mae: 0.0438 - mse: 0.0061 - val_loss: 0.2171 - val_mae: 0.0432 - val_mse: 0.0060
Epoch 153/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2169 - mae: 0.0437 - mse: 0.0061 - val_loss: 0.2165 - val_mae: 0.0432 - val_mse: 0.0059
Epoch 154/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2164 - mae: 0.0435 - mse: 0.0061 - val_loss: 0.2160 - val_mae: 0.0431 - val_mse: 0.0059
Epoch 155/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2158 - mae: 0.0435 - mse: 0.0060 - val_loss: 0.2154 - val_mae: 0.0429 - val_mse: 0.0059
Epoch 156/1000
60/6

60/60 [==============================] - 0s 6ms/step - loss: 0.1835 - mae: 0.0404 - mse: 0.0054 - val_loss: 0.1829 - val_mae: 0.0399 - val_mse: 0.0053
Epoch 200/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.1826 - mae: 0.0404 - mse: 0.0054 - val_loss: 0.1822 - val_mae: 0.0408 - val_mse: 0.0054
Epoch 201/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.1818 - mae: 0.0404 - mse: 0.0054 - val_loss: 0.1812 - val_mae: 0.0399 - val_mse: 0.0052
Epoch 202/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.1809 - mae: 0.0403 - mse: 0.0054 - val_loss: 0.1825 - val_mae: 0.0479 - val_mse: 0.0074
Epoch 203/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.1803 - mae: 0.0410 - mse: 0.0056 - val_loss: 0.1795 - val_mae: 0.0398 - val_mse: 0.0052
Epoch 204/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.1793 - mae: 0.0403 - mse: 0.0054 - val_loss: 0.1787 - val_mae: 0.0398 - val_mse: 0.0052
Epoch 205/1000
60/6

...

60/60 [==============================] - 0s 5ms/step - loss: 0.0066 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0064 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 788/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0066 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0064 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 789/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0066 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0064 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 790/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0066 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0064 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 791/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0066 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0064 - val_mae: 0.0446 - val_mse: 0.0060
Epoch 792/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0066 - mae: 0.0451 - mse: 0.0062 - val_loss: 0.0064 - val_mae: 0.0446 - val_mse: 0.0060
Epoch 793/1000
60/6

60/60 [==============================] - 0s 6ms/step - loss: 0.0064 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0063 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 837/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0064 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0063 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 838/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0064 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0063 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 839/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0064 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0445 - val_mse: 0.0060
Epoch 840/1000
60/60 [==============================] - 0s 7ms/step - loss: 0.0064 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 841/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0064 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 842/1000
60/6

60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 886/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 887/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 888/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0442 - val_mse: 0.0060
Epoch 889/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0445 - val_mse: 0.0060
Epoch 890/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0062 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 891/1000
60/6

60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0441 - val_mse: 0.0060
Epoch 935/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 936/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0445 - val_mse: 0.0060
Epoch 937/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 938/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 939/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 940/1000
60/6

60/60 [==============================] - 0s 5ms/step - loss: 0.0062 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 984/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 985/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0063 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 986/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0062 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 987/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0062 - mae: 0.0450 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0443 - val_mse: 0.0060
Epoch 988/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0062 - mae: 0.0449 - mse: 0.0062 - val_loss: 0.0061 - val_mae: 0.0444 - val_mse: 0.0060
Epoch 989/1000
60/6

In [34]:
model.evaluate([x_train,basis_train],y_train)

1844/1844 [==============================] - 3s 2ms/step - loss: 0.0062 - mae: 0.0450 - mse: 0.0062


[0.006245328113436699, 0.044989023357629776, 0.0061887288466095924]

In [35]:
model.evaluate([x_val,basis_val],y_val)

461/461 [==============================] - 1s 2ms/step - loss: 0.0061 - mae: 0.0444 - mse: 0.0060


[0.0060745058581233025, 0.04441540688276291, 0.006017883773893118]

In [36]:
save = model.save("/home/haoyan/Desktop/ML-CFD/covdiv-pope.h5")